In [6]:
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os

# Load dataset
csv_path = r"D:\1.UNI 5th Semester\6. Advance DataBase\ADVANCE_DB PROJECT - Copy (3)\ds_project\ruthapp\ml\Ruth_OLAP.csv"
df = pd.read_csv(csv_path)

# Fitur engineering
df['duration_per_activity'] = df['total_duration_minutes'] / (df['total_activities'] + 1e-5)

# Kolom input dan target
features = ['gender', 'type_name', 'course_name', 'total_activities', 'total_duration_minutes', 'duration_per_activity', 'age']
target = 'grade'
df = df[features + [target]]
df = df.rename(columns={'type_name': 'activity_type'})

# One-hot encoding
X = pd.get_dummies(df.drop(columns=[target]))
y = df[target]

# Simpan kolom one-hot untuk preprocessing nanti
expected_columns = X.columns.tolist()

# Buat folder output jika belum ada
output_dir = os.path.join('ruthapp', 'ml')
os.makedirs(output_dir, exist_ok=True)

# Simpan expected_columns.pkl
joblib.dump(expected_columns, os.path.join(output_dir, 'expected_columns.pkl'))

# Train test split dan training model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = XGBRegressor()
model.fit(X_train, y_train)

# Evaluasi
y_pred = model.predict(X_test)
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")

# Simpan model
joblib.dump(model, os.path.join(output_dir, 'xgb_classifier_model.pkl'))


MSE: 286.6443786621094
R2 Score: -0.3692888021469116


['ruthapp\\ml\\xgb_classifier_model.pkl']

In [7]:
import pandas as pd
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import os

# Load dataset
csv_path = r"D:\1.UNI 5th Semester\6. Advance DataBase\ADVANCE_DB PROJECT - Copy (3)\ds_project\ruthapp\ml\Ruth_OLAP.csv"
df = pd.read_csv(csv_path)

# Feature engineering
df['duration_per_activity'] = df['total_duration_minutes'] / (df['total_activities'] + 1e-5)

# Kolom input dan target
features = ['gender', 'type_name', 'course_name', 'total_activities', 'total_duration_minutes', 'duration_per_activity', 'age']
df = df[features + ['grade']]
df = df.rename(columns={'type_name': 'activity_type'})

# Konversi target: grade (float) → label kelas
def to_label(score):
    if score < 60:
        return "Low"
    elif score < 75:
        return "Medium"
    elif score < 90:
        return "High"
    else:
        return "Excellent"

df['grade_label'] = df['grade'].apply(to_label)

# One-hot encoding untuk input
X = pd.get_dummies(df.drop(columns=['grade', 'grade_label']))
y = df['grade_label']

# Simpan expected_columns untuk preprocessing nanti
expected_columns = X.columns.tolist()

# Output directory
output_dir = os.path.join('ruthapp', 'ml')
os.makedirs(output_dir, exist_ok=True)

# Simpan expected_columns
joblib.dump(expected_columns, os.path.join(output_dir, 'model2_columns.pkl'))

# Split dan train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# Evaluasi
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Simpan model
joblib.dump(clf, os.path.join(output_dir, 'model2.pkl'))


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3], got ['Excellent' 'High' 'Low' 'Medium']